In [1]:
import gensim
import re, unicodedata, logging, string
import time

# Using Word embeddings (word2vec) to automagically fill the gaps
* Spanish model trained with 80M from wikipedia


In [2]:
PATH='/home/enriqueotero/datasets/nlp/wikipedia/data/'

# wikipedia_es_small > 80M words
fileToRead = PATH+'wikipedia_es_small.xml'

In [ ]:
start = time.time()

print "File:"
print fileToRead

def clean(x):
   x = unicodedata.normalize('NFKD', x).encode('ascii','ignore').lower()
   replace_punctuation = string.maketrans(string.punctuation, ' '*len(string.punctuation))
   x = x.translate(replace_punctuation)
   x = re.sub('@%$&[\n/:!,;)()_?¿¡<>]', ' ', x)
   x = re.sub(' - ', ' ', x)
   x = re.sub(' +',' ', x).strip()
   return x
    
sentences = []
with open(fileToRead, 'r') as fileData:
    for lineas in fileData:
        #Formatear linea si hace falta aquí
        lineArray = lineas.split(".")
        for line in lineArray:
            if len(line) > 1:
                line = line.decode('utf-8')
                line = clean(line) # ¿? problemas con gensim y tildes y eñes...
                if len(line) > 1:
                    sentences.append(line.split(" "))
    fileData.close()


print "\nNumber of sentences"             
print len(sentences)

done = time.time()
elapsed = done - start
print(elapsed)

In [ ]:
start = time.time()

model = gensim.models.Word2Vec(sentences, size=100, workers=3, negative=1, sg=1)

done = time.time()
elapsed = done - start
print(elapsed)

### Save model

In [ ]:
gensim.models.Word2Vec.save(model,'wikipedia_model_es')

### And load model again

In [3]:
# And load model again
FILENAME = 'wikipedia_model_es'
model = gensim.models.Word2Vec.load(FILENAME)

### Try some typical phrases

In [4]:
words = "el rey juan carlos y la reina".split()
print(model.predict_output_word(words))

[('rey', 0.00019195296), ('ii', 0.00015460837), ('iii', 0.00011868595), ('iv', 0.00011611087), ('i', 0.00011187967), ('duque', 0.00010843632), ('principe', 0.00010823992), ('felipe', 0.00010768851), ('conde', 0.00010714246), ('castilla', 0.00010646953)]


Remove existing words

In [5]:
words = "el rey juan carlos y la reina".split()
output = model.predict_output_word(words)
output = [x for x in output if x[0] not in words]
print output

[('ii', 0.00015460837), ('iii', 0.00011868595), ('iv', 0.00011611087), ('i', 0.00011187967), ('duque', 0.00010843632), ('principe', 0.00010823992), ('felipe', 0.00010768851), ('conde', 0.00010714246), ('castilla', 0.00010646953)]


In [6]:
words = "buenas y hasta mañana".split()
output = model.predict_output_word(words)
output = [x for x in output if x[0] not in words]
print output

[('temporadas', 7.6493874e-05), ('muy', 6.8064357e-05), ('desde', 6.0666782e-05), ('mantuvo', 5.8725513e-05), ('ventas', 5.7498029e-05), ('sus', 5.675741e-05), ('pero', 5.4015534e-05), ('tierras', 5.2203679e-05)]


In [7]:
words = "informática y lenguajes de".split()
output = model.predict_output_word(words)
output = [x for x in output if x[0] not in words]
print output

[('programacion', 9.2945374e-05), ('otros', 8.1234597e-05), ('sistemas', 7.485925e-05), ('software', 7.3789241e-05), ('son', 7.3208372e-05), ('archivos', 7.3128169e-05), ('algunos', 7.117918e-05), ('java', 6.9016751e-05), ('datos', 6.9002206e-05)]


### Try actual fill-the-gap questions. 
Generated with `wikitrivia --lang es 'Fernando VII'`

In [8]:
words = "La última fase de su reinado, la llamada Década Ominosa, se caracterizó por una feroz de los exaltados, acompañada de una política absolutista moderada o incluso liberaldoctrinaria que provocó un profundo descontento en los círculos absolutistas, que formaron partido en torno al infante Carlos María Isidro".split()
print(model.predict_output_word(words))

[('parte', 5.6180288e-05), ('lo', 5.1669511e-05), ('mas', 4.5763965e-05), ('contra', 4.5105324e-05), ('partido', 4.4790893e-05), ('pero', 4.4231358e-05), ('sobre', 4.3421747e-05), ('era', 4.308984e-05), ('gran', 4.1953499e-05), ('frente', 4.17138e-05)]


In [9]:
words = "La última fase de su reinado, la llamada Década Ominosa, se caracterizó por una feroz de los exaltados, acompañada de una política absolutista moderada o incluso liberaldoctrinaria que provocó un profundo descontento en los círculos absolutistas, que formaron partido en torno al infante Carlos María Isidro".split()
output = model.predict_output_word(words, topn=1000000)
print len(output)
output = [x for x in output if x[0] in ["represion", "parada", "descalificacion", "negacion", "proyeccion"]]
print output

233361
[('represion', 2.8628981e-05), ('parada', 1.4828272e-05), ('proyeccion', 1.1659075e-05), ('negacion', 1.0614357e-05), ('descalificacion', 5.8318933e-06)]


In [10]:
words = "una sin escrupulos vengativa y traicionera".split()
output = model.predict_output_word(words, topn=1000000)
output = [x for x in output if x[0] in ["persona, amenaza, amistad, fauna, flora"]]
print output

[]


In [11]:
import random

In [12]:
words = "una feroz de los exaltados".split()
output = model.predict_output_word(words, topn=1000000)
print len(output)
choices = ["represion", "parada", "descalificacion", "negacion", "proyeccion"]
output = [x for x in output if x[0] in choices]
if len(output) == 0:
    output = random.choice(choices)
print output

233361
[('represion', 4.7717975e-05), ('parada', 1.4552617e-05), ('negacion', 1.3123062e-05), ('proyeccion', 9.3422059e-06), ('descalificacion', 4.9355567e-06)]


In [13]:
import json
jsonfile = 'data/historia_i.json'

def json_to_dict(filename):
    with open(filename) as json_data:
        d = json.load(json_data)
    return d

d = json_to_dict(jsonfile)
N = len(d)
print "Analyzing {} questions".format(N)

Analyzing 31 questions


In [17]:
from random import shuffle

def get_correct_answers(d):
    correct = 0.0
    for question in d:

        answer = "WRONG! "
        myquestion = question["question"].replace("__________", "{}")
   
        choices = [question["answer"]]
        choices.extend(question["similar_words"])
    
        output = model.predict_output_word(words, topn=1000000)
        output = [x for x in output if x[0] in choices]
        if len(output) == 0:
            output = random.choice(choices)
        else:
            output = output[0]
        #print output[0]
       
        if (output[0] == choices[0]):
            correct+=1
            answer = "RIGHT! "

    #print "Choices: {}".format(map(str,choices))
    
    return correct

correct = get_correct_answers(d)

In [18]:
print "Accuracy = %i/%i = %.2f%%" %(correct, N, 100*correct/N)

Accuracy = 11/31 = 35.48%
